In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [11]:
df["timestamp_bin"] = pd.cut(df["timestamp"], [-1, 10**4*5, 10**5, 10**5*5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])

In [12]:
df.groupby(["timestamp_bin"])["answered_correctly"].describe().head(200)

,count,mean,std,min,25%,50%,75%,max
timestamp_bin,,,,,,,,
"(-1, 50000]",93093.0,0.584190,0.492864,0.0,0.0,1.0,1.0,1.0
"(50000, 100000]",66825.0,0.492465,0.499947,0.0,0.0,0.0,1.0,1.0
"(100000, 500000]",355121.0,0.449306,0.497424,0.0,0.0,0.0,1.0,1.0
"(500000, 1000000]",214733.0,0.506666,0.499957,0.0,0.0,1.0,1.0,1.0
"(1000000, 10000000]",268824.0,0.619602,0.485485,0.0,0.0,1.0,1.0,1.0
"(10000000, 100000000]",410041.0,0.646482,0.478062,0.0,0.0,1.0,1.0,1.0
"(100000000, 1000000000]",1870769.0,0.662532,0.472846,0.0,0.0,1.0,1.0,1.0
"(1000000000, 10000000000]",4195074.0,0.673581,0.468903,0.0,0.0,1.0,1.0,1.0
"(10000000000, 100000000000]",2363062.0,0.673409,0.468966,0.0,0.0,1.0,1.0,1.0


In [10]:
df[df["timestamp"] < 100000]

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags,tag,type_of,timestamp_bin
0,0,24600,7900,0,0,0,1.0,NaN,NaN,7900,7900,0,1,131 93 81,-1,NaN,"(-1, 100000]"
1,25379,24600,7876,0,1,2,0.0,24000.0,False,7876,7876,3,1,10 94 92,-1,NaN,"(-1, 100000]"
2,50137,24600,175,0,2,2,1.0,23000.0,False,175,175,2,1,9 10 92,-1,NaN,"(-1, 100000]"
3,70181,24600,1278,0,3,3,1.0,22000.0,False,1278,1278,3,2,143 140 81 29,-1,NaN,"(-1, 100000]"
50,0,108310,3734,0,0,2,0.0,NaN,NaN,3734,3734,3,5,125,-1,NaN,"(-1, 100000]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10033817,22971,2147481750,1270,0,1,0,0.0,19000.0,False,1270,1270,3,2,143 71 38 29,-1,NaN,"(-1, 100000]"
10033818,33539,2147481750,9261,0,2,0,1.0,18000.0,False,9261,9261,0,5,173,-1,NaN,"(-1, 100000]"
10033819,57284,2147481750,8201,0,3,0,1.0,8000.0,False,8201,8201,0,5,8,-1,NaN,"(-1, 100000]"
10033820,75162,2147481750,367,0,4,0,1.0,21000.0,False,367,367,0,2,143 62 176 92 102,-1,NaN,"(-1, 100000]"
